In [2]:
!pip install mediapipe==0.10.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


In [3]:
import cv2
import numpy as np
import mediapipe as mp
import cloudpickle
from tensorflow.keras.models import load_model

In [4]:
# Load preprocessor and model
with open("/content/drive/MyDrive/Colab Notebooks/Yoga-82/new_preprocessor.pkl", 'rb') as f:  # Open the file in binary read mode ('rb')
    preprocessor = cloudpickle.load(f)  # Load from the file object
model = load_model("/content/drive/MyDrive/Colab Notebooks/Yoga-82/checkpoint.h5")

In [5]:
import numpy as np
import pandas as pd

def preprocess_first(X: pd.Series) -> np.ndarray:
    return np.vstack(X.apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))).astype(np.float64)

In [6]:
def select_relevant_landmarks(X: np.ndarray, relevant_ids=None) -> np.ndarray:
    if relevant_ids is None:
        relevant_ids = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28, 31, 32]

    selected_indices = []
    for idx in relevant_ids:
        selected_indices.extend([idx * 3, idx * 3 + 1, idx * 3 + 2])

    return X[:, selected_indices]

In [7]:
mp_pose = mp.solutions.pose

def extract_pose_landmarks(image):
    with mp_pose.Pose(static_image_mode=True) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            pose_array = np.array([[lm.x, lm.y, lm.z] for lm in landmarks]).flatten()
            return pose_array  # Shape: (99,)
        else:
            return None

In [13]:
# Load your image
image = cv2.imread("/content/download.jpg")

# Get 99 keypoints
pose_vector = extract_pose_landmarks(image)

In [14]:
if pose_vector is not None:
    # Convert pose_vector to a string representation of an array
    pose_vector_str = np.array2string(pose_vector, separator=' ', suppress_small=True)
    pose_vector_series = pd.Series([pose_vector_str]) # Create a Series with the string representation
    # Predict
    processed = preprocessor.transform(pose_vector_series)
    prediction = model.predict(processed)
    predicted_class = np.argmax(prediction)

    print(f"Predicted class index: {predicted_class}")
else:
    print("No human pose detected.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Predicted class index: 16


In [15]:
import cloudpickle

label_encoder = cloudpickle.load(open("/content/drive/MyDrive/Colab Notebooks/Yoga-82/label_encoder.pkl", 'rb'))

In [16]:
pose_name = label_encoder.inverse_transform([predicted_class])[0]
print(f"Predicted pose: {pose_name}")

Predicted pose: Extended_Revolved_Triangle_Pose_or_Utthita_Trikonasana_
